In [ ]:
pip install PyPortfolioOpt
pip install pandas_datareader
pip install yfinance

In [ ]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from datetime import datetime
import math
import os
import datetime
import xlsxwriter
import numpy as np
import pandas as pd
import pandas_datareader as pdr
import matplotlib.pyplot as plt
import yfinance as yf
import csv
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
yf.pdr_override()

In [ ]:
#Loading the Tickers as Identified by Global X-funds in their Fin-tech ETF
tk = pd.read_csv("/home/rampage/Downloads/global-x-cloud-computing-etf_full-holdings_20191008.csv")
tk.columns = ('Global X Cloud Computing ETF', 'Name', 'Price', 'Shares Held', 'Market Value')
tk = tk.drop([0,1], axis = 0)
companies = pd.DataFrame(tk['Name'].unique()).reset_index()[0]
start = datetime.datetime(2014,1,1)
end = datetime.datetime(2019,7,1)

In [ ]:
## Creating a range of dates over which to find the weights
dates = []
date = datetime.datetime(2018,7,16)
for i in range(52):
    date += datetime.timedelta(days = 7) 
    dates.append(date)

In [ ]:
## Function to find ticker from company name
import requests
def get_symbol(symbol):
    url = "http://d.yimg.com/autoc.finance.yahoo.com/autoc?query={}&region=1&lang=en".format(symbol)

    result = requests.get(url).json()

    for x in result['ResultSet']['Result']:
        return x['symbol']

In [ ]:
## Remove Unnecessary words from Company titles to find corresponding ticker values
stopwords = {'INC', 'Holding',' CORP', 'INC-CL A', 'INC-CLASS A', 'LTD', 'GRP-ADR','-', 'CLASS', 'A','-CL A','INC-CLASS','CORP','INC-ADR','INC-CL','TRUST', 'CASH' }
title =[]
for i in companies:
    rs = [word for word in i.split() if word not in stopwords]
    rs = ' '.join(rs)
    title.append(rs)

In [ ]:
#Get the Tickers from company titles
tickers = []
for i in title:
    com = get_symbol(i)
    tickers.append(com)

In [ ]:
## Defining Close function to get daily close prices of the stocks
def close(tickers, startdate, enddate):
    def data(ticker):
        try:
            return (pdr.get_data_yahoo(ticker, start = startdate, end = enddate)['Adj Close']).rename(ticker)
        except:
            pass
    datas = map(data, tickers)
    return(pd.concat(datas, axis = 1))
dataset = close(tickers, start, end)

In [ ]:
#Cleaning data to remove all empty rows
all_data = dataset.dropna(axis = 0, thresh = 3)
all_data = all_data.fillna(method='bfill')

#mean and covariance
mean = expected_returns.mean_historical_return(all_data)
sd = risk_models.sample_cov(all_data)

e_frontier = EfficientFrontier(mean, sd)
raw_weights = e_frontier.max_sharpe()
cleaned_weights = e_frontier.clean_weights()
e_frontier.portfolio_performance(verbose=True)

In [ ]:
## Big Lopp for getting efficient weights each of 52 weeks
alldata = []
#Getting Ajd Close for all the tickers
for i in dates:
    end = i
    dataset = close(tickers, start, end)
    all_data = dataset.dropna(axis = 0, thresh = 3)
    all_data = all_data.fillna(method='bfill')

#mean and covariance
    mu = expected_returns.mean_historical_return(all_data)
    S = risk_models.sample_cov(all_data)
    
    ef = EfficientFrontier(mu, S)

    raw_weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()
    
    f_dat = pd.DataFrame(cleaned_weights, index=[0])


    ret = ef.portfolio_performance(verbose=False)[0]
    vol = ef.portfolio_performance(verbose=False)[1]
    sharp = ef.portfolio_performance(verbose=False)[2]

    f_dat['Return'] = ret
    f_dat['Volatility'] = vol
    f_dat['Sharpe.Ratio'] = sharp
    
    alldata.append(f_dat)
    print(f_dat)

In [ ]:
## Aggregating the list entries in a dataframe
mod_data = []
for i in range(0,len(alldata)):
    new = alldata[i]
    mod_data.append(new)
mod_data = pd.concat(mod_data)

In [ ]:
mod_data.to_csv('out.csv')